In [ ]:
import sys
sys.path.append('..')

In [ ]:
from pathlib import Path

In [ ]:
import pandas as pd
import numpy as np
import cv2

In [ ]:
from lib.evaluate import Evaluator

In [ ]:
from PIL import Image
from IPython.display import display

def show(ary):
    display(Image.fromarray(ary))

In [ ]:
src = sorted(Path('/home/jiun/datasets/amano/cylinder/refine-annot/train/annotations').glob('*.txt'))

In [ ]:
detections = sorted(Path('/home/jiun/datasets/weights/train-self/test-vgg16/pseudo').iterdir())

In [ ]:
image = cv2.imread('/home/jiun/datasets/amano/cylinder/refine-annot/train/images/nvr001_10_CT_20190110035022+1.jpg')

In [ ]:
improt torch

In [ ]:
for detection in detections[::32]:
    det = sorted(detection.glob('*.txt'))
    
    evaluator = Evaluator(n_class=2)
    
    img = image.copy()
    
    for index, (gtt, dtt) in enumerate(zip(src, det)):
        gt = pd.read_csv(str(gtt), header=None).values
        dt = pd.read_csv(str(dtt), header=None).values
        
        if index == 0:
            for x, y, w, h in gt:
                cv2.rectangle(img, (int(x), int(y)), (int(w), int(h)), (0, 0, 255), 3)
                
            for x, y, w, h in dt:
                cv2.rectangle(img, (int(x), int(y)), (int(w), int(h)), (255, 0, 0), 3)
        
        evaluator.update((
            np.ones(np.size(dt, 0), dtype=np.int),
            np.ones(np.size(dt, 0), dtype=np.float32),
            dt,
            None,
        ), (
            np.ones(np.size(gt, 0), dtype=np.int),
            gt.astype(np.float32),
            None,
        ))
    
    for klass, (ap, precision, recall) in enumerate(zip(*evaluator.dump())):
        if klass == 0:
            continue
        
        print(f'Iter {detection.stem}: {ap:.4f}')
        show(img)